# **Json write ipynb**

- reference:  https://bleedai.com/introduction-to-pose-detection-and-basic-pose-classification/

In [19]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import json
from IPython.display import clear_output
import os
from PIL import Image

## **getGifAvgFps**

In [20]:
def getGifAvgFps(pil_ImageObject):
    """ Returns the average framerate of a PIL Image object """
    pil_ImageObject.seek(0)
    frames = duration = 0
    while True:
        try:
            frames += 1
            duration += pil_ImageObject.info['duration']
            pil_ImageObject.seek(pil_ImageObject.tell() + 1)
        except EOFError:
            return frames / duration * 1000
    return 24

## **Mediapipe landmark names**

In [21]:

name_list = [
 "nose"
,"left_eye_inner"
,"left_eye"
,"left_eye_outer"
,"right_eye_inner"
,"right_eye"
,"right_eye_outer"
,"left_ear"
,"right_ear"
,"mouth_left"
,"mouth_right"
,"left_shoulder"
,"right_shoulder"
,"left_elbow"
,"right_elbow"
,"left_wrist"
,"right_wrist"
,"left_pinky"
,"right_pinky"
,"left_index"
,"right_index"
,"left_thumb"
,"right_thumb"
,"left_hip"
,"right_hip"
,"left_knee"
,"right_knee"
,"left_ankle"
,"right_ankle"
,"left_heel"
,"right_heel"
,"left_foot_index"
,"right_foot_index"]



## **Mediapipe init**

In [22]:
mp_pose = mp.solutions.pose
 
# Setting up the Pose function.
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.6, model_complexity=2)
 
# Initializing mediapipe drawing class, useful for annotation.
mp_drawing = mp.solutions.drawing_utils

## **Mediapipe detect pose**

- This function performs pose detection on an image.
    - Args:
        - image: The input image with a prominent person whose pose landmarks needs to be detected.
        - pose: The pose setup function required to perform the pose detection.
        - display: A boolean value that is if set to true the function displays the original input image, the resultant image, 
       ' and the pose landmarks in 3D plot and returns nothing.
    - Returns:
        - output_image: The input image with the detected pose landmarks drawn.
        - landmarks: A list of detected landmarks converted into their original scale.

- Pose World landmarks
    - `x`: `-landmark[i].z`
    - `y`: `landmark[i].x` 
    - `z`: `-landmark[i].y`
    - `score`:  `landmark[i].visibility`
    - `name`: `name_list[i]`


In [23]:
def detectPose(image, pose, jsonObject, frameNum, display=True):
    clear_output(wait=True) # for clear console output

    # for json_object['frames']
    frameJsonObject = {
        "frameNum": frameNum,
        "keypoints": [
        ],
        "keypoints3D" : [
        ]
    }
    
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = pose.process(imageRGB)
    
    # Retrieve the height and width of the input image.
    height, width, _ = image.shape
    
    # Initialize a list to store the detected landmarks.
    landmarks = []

    if results.pose_world_landmarks:
        for i in range(0, 33):
            landmark = results.pose_world_landmarks.landmark
            frameJsonObject["keypoints3D"].append({'x': -landmark[i].z,'y': landmark[i].x, 'z': -landmark[i].y,'score':  landmark[i].visibility, "name": name_list[i]})
            
    if results.pose_landmarks:
        # Draw Pose landmarks on the output image.
        for i in range(0, 33):
            landmark = results.pose_landmarks.landmark
            frameJsonObject["keypoints"].append({'x': landmark[i].x,'y': landmark[i].y, 'z': landmark[i].z,'score':  landmark[i].visibility, "name": name_list[i]})

            
    mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                              connections=mp_pose.POSE_CONNECTIONS)


    jsonObject["frames"].append(frameJsonObject)
    # Check if the original input image and the resultant image are specified to be displayed.
    if display:
        # Display the original input image and the resultant image.
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
        
        # Also Plot the Pose landmarks in 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        
        
        # Return the output image and the found landmarks.
    return output_image, landmarks

## **Prepare**

In [24]:
fileName = 'sample/mixamo_T_pose.gif'

In [25]:
FILENAME = os.path.join(os.path.dirname('__file__'),fileName)
# only gif
fps = getGifAvgFps(Image.open(FILENAME))
jsonObject = {
    "fileName": fileName,
    "duration": 0,
    "ticksPerSecond": math.trunc(fps),
    "frames": [
    ]
}

In [26]:
# Setup Pose function for video.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)
 
# Initialize the VideoCapture object to read from the webcam.
#video = cv2.VideoCapture(0)
 
# Initialize the VideoCapture object to read from a video stored in the disk.
video = cv2.VideoCapture(fileName)
 
 
# Initialize a variable to store the time of the previous frame.
time1 = 0

frame_num = -1
 

## **Detect pose**

In [27]:
# Iterate until the video is accessed successfully.
while video.isOpened():
    
    # Read a frame.
    ok, frame = video.read()
    frame_num += 1
    
    # Check if frame is not read properly.
    if not ok:
        
        # Break the loop.
        break
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    # frame = cv2.flip(frame, 1)
    
    # Get the width and height of the frame
    frame_height, frame_width, _ =  frame.shape
    
    # Resize the frame while keeping the aspect ratio.
    frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
    
    # Perform Pose landmark detection.
    frame, _ = detectPose(frame, pose_video, jsonObject, frame_num, display=False)
    
    # Set the time for this frame to the current time.
    time2 = time()
    
    # Check if the difference between the previous and this frame time &gt; 0 to avoid division by zero.
    if (time2 - time1):
    
        # Calculate the number of frames per second.
        frames_per_second = 1.0 / (time2 - time1)
        
        # Write the calculated number of frames per second on the frame. 
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30),cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    
    # Update the previous frame time to this frame time.
    # As this frame will become previous frame in next iteration.
    time1 = time2
    
    # Display the frame.
    cv2.imshow('Pose Detection', frame)
    
    # Wait until a key is pressed.
    # Retreive the ASCII code of the key pressed
    k = cv2.waitKey(1)
    
    # Check if 'ESC' is pressed.
    if(k == 27):
        
        # Break the loop.
        break
jsonObject['duration'] = frame_num -1;
with open( fileName+'.json', 'w') as f:
    json_string = json.dump(jsonObject, f, indent=2)    
# Release the VideoCapture object.
video.release()
 
# Close the windows.
cv2.destroyAllWindows()